In [1]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession, Row
import pandas as pd


In [2]:
spark = SparkSession.builder.appName("rdd-app").config("spark.config.option", "value").getOrCreate()
scfg = SparkConf().setAppName('rdd-app')
sc = spark.sparkContext

In [3]:
import string

text_file = '/user/student/shakespeare/tragedy/hamlet.txt'
text = sc.textFile(text_file)

In [4]:
def strip_punc(s):
    return s.translate(str.maketrans('', '', string.punctuation)).split(' ')

def search_word_in_line(word):
    count = 1
    for line in text.collect():
        if word in strip_punc(line):
            print('{}. {}'.format(count, line))
        count += 1

In [5]:
flatmap = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))
map = flatmap.map(lambda word: (word, 1))
reduced = map.reduceByKey(lambda a, b: a + b)

In [6]:
counts = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))\
             .map(lambda word: (word, 1))\
             .reduceByKey(lambda a, b: a + b)    

In [7]:
word = "purpose"
for count in reduced.collect():
    # kv = str(count).translate(str.maketrans('', '', string.punctuation)).split(' ')
    kv = strip_punc(str(count))
    if word == kv[0]:
        print('Found \'{}\' occurs \'{}\' times'.format(kv[0], kv[1])) 
        search_word_in_line(word)
        break

Found 'purpose' occurs '11' times
2599.     Why, any thing, but to the purpose. You were sent
2926.     Black as his purpose, did the night resemble
3216.     And drive his purpose on to these delights.
3540.     from the purpose of playing, whose end, both at the
3909.     The passion ending, doth the purpose lose.
4766.     Is but to whet thy almost blunted purpose.
6202.     And, for that purpose, I'll anoint my sword.
6227.     Our purpose may hold there.
6378.     purpose, confess thyself--
7328.     king hold his purpose, I will win for him an I can;
7376.     I am constant to my purpose; they follow the king's


## Manipulating airline performance data

In [8]:

from pyspark.sql.types import Row
from datetime import datetime


In [9]:
data_by_year = '/user/student/airline/2003.csv'
airline_performance = spark.read.option("header", "true").csv(data_by_year)

In [10]:
airline_performance.head()

Row(Year='2003', Month='1', DayofMonth='29', DayOfWeek='3', DepTime='1651', CRSDepTime='1655', ArrTime='1912', CRSArrTime='1913', UniqueCarrier='UA', FlightNum='1017', TailNum='N202UA', ActualElapsedTime='141', CRSElapsedTime='138', AirTime='119', ArrDelay='-1', DepDelay='-4', Origin='ORD', Dest='MSY', Distance='837', TaxiIn='5', TaxiOut='17', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [11]:
airline_performance

DataFrame[Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [12]:
from pyspark.sql.types import IntegerType
airline_performance = airline_performance.withColumn("ArrDelay", airline_performance["ArrDelay"].cast(IntegerType()))
airline_performance = airline_performance.withColumn("DepDelay", airline_performance["DepDelay"].cast(IntegerType()))

In [13]:
airline_performance.describe(['ArrDelay']).show()
airline_performance.describe(['DepDelay']).show()

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|           6375689|
|   mean| 3.596694098473122|
| stddev|30.315633327428507|
|    min|              -937|
|    max|              1612|
+-------+------------------+

+-------+------------------+
|summary|          DepDelay|
+-------+------------------+
|  count|           6387071|
|   mean|  5.24823052695046|
| stddev|26.170174590947493|
|    min|             -1410|
|    max|              1582|
+-------+------------------+



In [14]:
import pyspark.sql.functions as F
airline_performance.select(airline_performance.Dest,F.when(airline_performance.ArrDelay > 0, 1).otherwise(0)).show()

+----+------------------------------------------+
|Dest|CASE WHEN (ArrDelay > 0) THEN 1 ELSE 0 END|
+----+------------------------------------------+
| MSY|                                         0|
| MSY|                                         0|
| MSY|                                         1|
| ORD|                                         0|
| ORD|                                         1|
| ORD|                                         1|
| ORD|                                         0|
| ORD|                                         1|
| ORD|                                         1|
| BOS|                                         1|
| BOS|                                         0|
| BOS|                                         0|
| BOS|                                         0|
| BOS|                                         1|
| BOS|                                         0|
| BOS|                                         0|
| BOS|                                         0|


In [15]:
airline_performance = airline_performance.withColumn('ArrDelayCount',F.when(airline_performance.ArrDelay > 0, 1).otherwise(0))
airline_performance = airline_performance.withColumn('DepDelayCount',F.when(airline_performance.DepDelay > 0, 1).otherwise(0))
airline_performance.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCount|DepDelayCount|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|2003|    

In [16]:
#Most Depature Delay Airport 2003
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+------+------+
|Year|Origin| count|
+----+------+------+
|2003|   ATL|167354|
+----+------+------+
only showing top 1 row



In [17]:
#Least Depature Delay Airport 2003
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+------+-----+
|Year|Origin|count|
+----+------+-----+
|2003|   BFF|    1|
+----+------+-----+
only showing top 1 row



In [18]:
#Most Arrival Delay Airport 2003
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+----+------+
|Year|Dest| count|
+----+----+------+
|2003| ATL|153285|
+----+----+------+
only showing top 1 row



In [19]:
#least Arrival Delay Airport 2003
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+----+-----+
|Year|Dest|count|
+----+----+-----+
|2003| MKK|    5|
+----+----+-----+
only showing top 1 row



In [20]:
#Most Arrival Delay Flight 2003
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2003|           WN|     2169|  963|
+----+-------------+---------+-----+
only showing top 1 row



In [21]:
#Least Arrival Delay Flight 2003
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2003|           XE|     3167|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [22]:
#Most Departure Delay Flight 2003
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2003|           AS|       64|  858|
+----+-------------+---------+-----+
only showing top 1 row



In [23]:
#Least Departure Delay Flight 2003
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2003|           UA|     1939|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [24]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('ArrDelay'))\
.show()

+----+-------------+--------------------+
|Year|UniqueCarrier|       avg(ArrDelay)|
+----+-------------+--------------------+
|2003|           MQ|   4.239333625771869|
|2003|           XE|   5.228967480333107|
|2003|           CO|   3.782939738463129|
|2003|           AS|  3.1776754102105302|
|2003|           HP|   3.061532792925571|
|2003|           EV|   6.659896536212326|
|2003|           NW|  1.9632481097925139|
|2003|           WN|  2.0638629859031217|
|2003|           HA|-0.14296714579055442|
|2003|           FL|   8.366069865632243|
|2003|           DH|   6.533820220706404|
|2003|           US|   4.223097968334864|
|2003|           OO|  2.2876037171402035|
|2003|           AA|    3.20733799370944|
|2003|           UA|   3.113908238982237|
|2003|           DL|  3.8552977159126085|
|2003|           TZ|   4.959598623013258|
|2003|           B6|   0.986033686986753|
+----+-------------+--------------------+



In [25]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('DepDelay'))\
.show()

+----+-------------+------------------+
|Year|UniqueCarrier|     avg(DepDelay)|
+----+-------------+------------------+
|2003|           MQ| 5.028931816814106|
|2003|           XE| 4.138393896770467|
|2003|           CO| 3.900224032383165|
|2003|           AS| 5.798805902182644|
|2003|           HP| 5.823276597559935|
|2003|           EV|  8.88576272328195|
|2003|           NW|2.1861469315236306|
|2003|           WN| 6.702967177039947|
|2003|           HA|1.0343854246856556|
|2003|           FL| 6.906482814531121|
|2003|           DH| 9.403595306998161|
|2003|           US|  4.17614485278527|
|2003|           OO| 5.445892902895209|
|2003|           AA| 4.674403271720612|
|2003|           UA| 4.669889487359629|
|2003|           DL| 4.549610356232617|
|2003|           TZ| 4.788659416577627|
|2003|           B6|  5.65661821879299|
+----+-------------+------------------+

